In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from collections import deque
import time
import seaborn as sns
from tqdm import tqdm
from matplotlib.patches import Patch
import os
import pygame

In [2]:
# Set device for training (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the QNet class
class REINFORCE(nn.Module):
    def __init__(self, env, lr=0.005, device=device):
        super(REINFORCE, self).__init__()
        
        # Set device for training (GPU if available)
        self.device = device
        
        # Get state and action space dimensions
        self.state_space_dim = env.observation_space.shape[0]
        self.action_space_dim = env.action_space.n
        
        # Define possible actions
        self.actions = torch.arange(self.action_space_dim).to(device)
        
        # Set learning rate
        self.lr = lr
        
        # Define neural network architecture
        self.net = nn.Sequential(
            nn.Linear(self.state_space_dim, 16, bias=True),
            nn.Tanh(),
            nn.Linear(16, 32, bias=True),
            nn.Tanh(),
            nn.Linear(32, self.action_space_dim, bias=True),
            nn.Softmax(dim=-1),
            )
        
        # Define optimizer
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)
    
    
    def forward(self, x):
        # Forward pass through the network
        return self.net(x.to(self.device))
    
    
    # Choose action based on epsilon-greedy policy
    def act(self, state):
        # Get action probabilities
        action_probs = self.forward(state)
        
        # Choose action based on action probabilities 
        action = torch.multinomial(action_probs, 1).item()
        
        return action
        
    def update_policy(self, rewards, log_probs, gamma):
        # Compute discounted rewards going backwards from the last state of the episode
        discounted_rewards = []
        R = 0
        
        for r in rewards[::-1]:    
            # Reset reward sum if we encounter a non-zero reward
  
            R = r + gamma * R
            
            # Insert discounted reward at the beginning of the list
            discounted_rewards.insert(0, R)
    
        # Convert discounted rewards to tensor and pass to device
        discounted_rewards = torch.tensor(discounted_rewards, dtype=torch.float32).to(self.device)
        
        #Normalize discounted rewards
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
        
        #List to store policy gradient loss 
        policy_gradient_loss = []
        
        # Compute policy gradient loss for each step of the episode
        for log_prob, reward in zip(log_probs, discounted_rewards):
            
            # append the negative of the policy gradient loss to the list
            policy_gradient_loss.append(-log_prob * reward)
        
        #Stack the sum of the policy gradient losses of this episode to the list of policy gradient loss    
        loss = torch.stack(policy_gradient_loss).sum()
        
        
        # Reset gradients to zero
        self.optimizer.zero_grad() 
        
        # Backpropagate the loss
        loss.backward()
        
        # Perform the optimization step
        self.optimizer.step()
        
        return loss.item() # Return the loss as a float for logging
        
    def save_model(self, filename):
        # Save model to file
        torch.save(self.state_dict(), filename)
    
    def load_model(self, filename, device='cuda'):
        # Load model from file
        self.load_state_dict(torch.load(filename, map_location=device))
        


In [5]:

import wandb
import os

# Initialize Weights & Biases
wandb.init(project="reinforce_lunar_lander", entity="ai42")


env = gym.make('LunarLander-v2', continuous=False, render_mode='rgb_array')
REINFORCE = REINFORCE(env).to(device)
optimizer = optim.Adam(REINFORCE.parameters(), lr=0.005)

#HYPERPARAMETERS
lr=0.005
gamma= 0.99 
batch_size= 8
max_episodes= 5000


# Define a video directory
video_dir = '/home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos' # Set this to your preferred directory
os.makedirs(video_dir, exist_ok=True)

# Wrap your environment to record videos
env = gym.wrappers.RecordVideo(env, video_folder=video_dir, episode_trigger=lambda episode_id: True)

# Set up lists to hold results
all_rewards = []
avg_rewards = []
losses = []

# Replace 'max_episodes' with the actual number
for episode in range(max_episodes):
    state, info = env.reset()
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)

    log_probs = []
    rewards = []
    total_reward = 0

    for t in range(1000):  # Limit the number of timesteps per episode
        action = REINFORCE.act(state)
        next_state, reward, terminated, truncated, info = env.step(action)
        next_state = torch.from_numpy(next_state).float().unsqueeze(0).to(device)

        # Store log probability and reward
        log_prob = torch.log(REINFORCE.forward(state)[0][action])
        log_probs.append(log_prob)
        rewards.append(reward)

        total_reward += reward
        state = next_state

        if terminated or truncated:
            break
     
    # Update policy after each episode
    loss = REINFORCE.update_policy(rewards, log_probs, gamma)
    losses.append(loss)

    # Store total reward
    all_rewards.append(total_reward)
    avg_reward = np.mean(all_rewards[-100:])  # Average reward of the last 100 episodes
    avg_rewards.append(avg_reward)

    wandb.log({"Episode Reward": total_reward, "Average Reward": avg_reward, "Loss": loss})   


    # After each episode, log the video file to wandb
    video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
    if video_files:
        last_video_file = video_files[-1]  # Assuming the last file in the list is the most recent
        wandb.log({"episode_video": wandb.Video(os.path.join(video_dir, last_video_file), fps=4, format="mp4")})

    # Optionally, clear the videos directory after logging
    for file in video_files:
        os.remove(os.path.join(video_dir, file))

    # Print progress
    if episode % 100 == 0:
        print(f"Episode {episode}, Average Reward: {avg_reward}, Loss: {loss}")

# Optionally, save the trained model
REINFORCE.save_model('reinforce_lunar_lander.pth')

# Plotting results
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
plt.plot(all_rewards, label='Total Reward')
plt.plot(avg_rewards, label='Average Reward')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Episode Rewards')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(losses)
plt.xlabel('Episode')
plt.ylabel('Loss')
plt.title('Policy Gradient Loss')
plt.show()
wandb.close()
env.close()
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: neildlf (ai42). Use `wandb login --relogin` to force relogin


/home/ndelafuente/miniconda3/envs/pytorch-env/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-0.mp4


Episode 0, Average Reward: -249.17532670908642, Loss: 1.367361068725586
Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-1.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-1.mp4



...

Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-663.mp4
Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-664.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-664.mp4



Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-664.mp4


Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-665.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-665.mp4



Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-665.mp4
Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-666.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-666.mp4



Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videos/rl-video-episode-666.mp4
